In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request as ur
import matplotlib.pyplot as plt
from matplotlib import style
import pickle

### Function to get the page-source of the company's url

In [2]:
def get_page_source(url):
    with ur.urlopen(url) as page_source:
        return page_source.read()

### Function to scrape the page and return the data as a list

In [3]:
def scraping(company):
    scrape_data = []
    page_url = "https://markets.financialcontent.com/stocks/quote/historical?Symbol=%s&Month=6&Year=2018&Range=12"
    source = get_page_source(page_url % company)
    soup = bs.BeautifulSoup(source, "lxml")
    table_data = soup.find("table", class_="quote_detailed_price_table data").find_all("tr")
    for row in table_data:
        r_data = row.find_all("td")
        scrape_data.append([data.text for data in r_data])
    return scrape_data

### Function to create the DataFrame with the scraped_data

In [4]:
def create_dframe(scrape_data):
    return pd.DataFrame(scrape_data, columns=["Date", "Open", "High", "Low", "Close", "Volume", "Change(%)"]).set_index("Date")

### Funtion to store the DataFrame as .csv file

In [5]:
def to_csv(frame, name):
    frame.to_csv(f"{name}_data.csv", encoding="utf-8")

### Function to remove the commas from Volume, change data type to float and remove all NaN values

In [6]:
def data_cleaning(company):
    company["Volume"] = company["Volume"].str.replace(",", "").astype(float)
    company.dropna(how="any")

In [7]:
companies = ("AAPL", "GOOGL", "MSFT")

### Get the page-source, create DataFrame and store the data as .csv

In [8]:
for company in companies:
    scrape_data = scraping(company)
    frame = create_dframe(scrape_data[1:])
    to_csv(frame, company)

### Get the from local storage to do further analysis

In [9]:
apple = pd.read_csv("AAPL_data.csv", index_col="Date")
google = pd.read_csv("GOOGL_data.csv", index_col="Date")
microsoft = pd.read_csv("MSFT_data.csv", index_col="Date")

In [10]:
companies = (apple, google, microsoft)

### Converting Volume to float and removing all NaN values

In [11]:
for company in companies:
    data_cleaning(company)